In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import zipfile
import datetime
import os

In [2]:
bl_old=pd.read_csv(r'C:\Users\User\OneDrive - ООО «Эдвансд»\for Power BI\smoothened_total_forecast_ML141221.dat', sep='|')  
bl_new=pd.read_csv(r'C:\Users\User\OneDrive - ООО «Эдвансд»\for Power BI\smoothened_total_forecast_ML130122.dat', sep='|')
df_sales=pd.read_csv('DOMO_UPC_SALES EXPORT.csv', dtype={'PA_DIRECT_CD': str})
df_plans=pd.read_csv('DOMO_UPC_PLANS EXPORT.csv', dtype={'PAD':str, 'PAI':str})
df_m=pd.read_excel('DHUB PH Extract.xlsx')

C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\User\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\User\anaconda3\lib\site-package

In [2]:
df_cust=pd.read_excel('DHUB CH Extract.xlsx', dtype={'CUST_ID_NR_CD':str, 'IND_ACCT_NR_CD':str})        
df_temp2 = pd.read_csv('Report PBI.csv', dtype={'CUST_ID_NR_CD':str, 'IND_ACCT_NR_CD':str})

C:\Users\User\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Baseline

In [3]:
bl_new.head()

,CUST_ID_NR_CD,IND_ACCT_NR_CD,PROMO_PLN_ID_NR_CD,PROMO_PLN_YR_CD,BRND_NR_CD,CS_UPC_NR_CD,CS_UPC_VRSN_NR_CD,WEEK_NR,SOURCE_CD,SUPPLEMENTAL_VOLUME_NR,CAT_NR_CD,FMLY_NR_CD,ACCT_PLAN_ID,DATE_LAST_UPDATED,DEL_IND,NSV_AMT,Unnamed: 16
0,386,0,NaN,2022,NaN,7940044778,NaN,18,NaN,1.731852,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,386,0,NaN,2022,NaN,7940044778,NaN,19,NaN,4.819204,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,386,0,NaN,2022,NaN,7940044778,NaN,20,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,386,0,NaN,2022,NaN,7940044778,NaN,21,NaN,0.981675,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,386,0,NaN,2022,NaN,7940044778,NaN,22,NaN,2.030436,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
bl_new.drop(columns=['PROMO_PLN_ID_NR_CD', 'BRND_NR_CD', 'CS_UPC_VRSN_NR_CD', 'SOURCE_CD', 'CAT_NR_CD', 'FMLY_NR_CD', 'ACCT_PLAN_ID', 'DEL_IND', 'NSV_AMT', 'Unnamed: 16', 'DATE_LAST_UPDATED'], inplace=True)
bl_old.drop(columns=['PROMO_PLN_ID_NR_CD', 'BRND_NR_CD', 'CS_UPC_VRSN_NR_CD', 'SOURCE_CD', 'CAT_NR_CD', 'FMLY_NR_CD', 'ACCT_PLAN_ID', 'DEL_IND', 'NSV_AMT', 'Unnamed: 16', 'DATE_LAST_UPDATED'], inplace=True)

In [5]:
bl_old.rename(columns={'SUPPLEMENTAL_VOLUME_NR': 'SUPPLEMENTAL_VOLUME_NR_old'}, inplace=True)

In [6]:
bl_old.columns

Index(['CUST_ID_NR_CD', 'IND_ACCT_NR_CD', 'PROMO_PLN_YR_CD', 'CS_UPC_NR_CD',
       'WEEK_NR', 'SUPPLEMENTAL_VOLUME_NR_old'],
      dtype='object')

In [7]:
bl_new['CUST_ID_NR_CD']=bl_new['CUST_ID_NR_CD'].astype(str).str.zfill(6)
bl_new['IND_ACCT_NR_CD']=bl_new['IND_ACCT_NR_CD'].astype(str).str.zfill(6)
bl_new['CS_UPC_NR_CD']=bl_new['CS_UPC_NR_CD'].astype(str).str.zfill(12)
bl_old['CUST_ID_NR_CD']=bl_old['CUST_ID_NR_CD'].astype(str).str.zfill(6)
bl_old['IND_ACCT_NR_CD']=bl_old['IND_ACCT_NR_CD'].astype(str).str.zfill(6)
bl_old['CS_UPC_NR_CD']=bl_old['CS_UPC_NR_CD'].astype(str).str.zfill(12)
bl_new1=bl_new.groupby(['CUST_ID_NR_CD', 'IND_ACCT_NR_CD', 'PROMO_PLN_YR_CD', 'CS_UPC_NR_CD','WEEK_NR']).agg({'SUPPLEMENTAL_VOLUME_NR': 'sum'})
bl_new1=bl_new1.reset_index()
bl_old1=bl_old.groupby(['CUST_ID_NR_CD', 'IND_ACCT_NR_CD', 'PROMO_PLN_YR_CD', 'CS_UPC_NR_CD','WEEK_NR']).agg({'SUPPLEMENTAL_VOLUME_NR_old': 'sum'})
bl_old1=bl_old1.reset_index()
bl_new1=bl_new.groupby(['CUST_ID_NR_CD', 'IND_ACCT_NR_CD', 'PROMO_PLN_YR_CD', 'CS_UPC_NR_CD','WEEK_NR']).agg({'SUPPLEMENTAL_VOLUME_NR': 'sum'})

bl_full_o=pd.merge(bl_new1, bl_old1, how='outer', on=['CUST_ID_NR_CD', 'IND_ACCT_NR_CD', 'PROMO_PLN_YR_CD', 'CS_UPC_NR_CD','WEEK_NR'])

In [8]:
bl_full_o=bl_full_o.drop_duplicates()


In [9]:
bl_full_o[bl_full_o.duplicated()]

,CUST_ID_NR_CD,IND_ACCT_NR_CD,PROMO_PLN_YR_CD,CS_UPC_NR_CD,WEEK_NR,SUPPLEMENTAL_VOLUME_NR,SUPPLEMENTAL_VOLUME_NR_old


In [10]:
bl_full_o.head()

,CUST_ID_NR_CD,IND_ACCT_NR_CD,PROMO_PLN_YR_CD,CS_UPC_NR_CD,WEEK_NR,SUPPLEMENTAL_VOLUME_NR,SUPPLEMENTAL_VOLUME_NR_old
0,000004,000000,2022,001111100120,2,0.0,22.837979
1,000004,000000,2022,001111100120,3,0.0,13.930047
2,000004,000000,2022,001111100120,4,0.0,12.818855
3,000004,000000,2022,001111100120,5,0.0,12.538084
4,000004,000000,2022,001111100120,6,0.0,12.538084


In [11]:
# bl_old.drop(columns=['PROMO_PLN_ID_NR_CD', 'BRND_NR_CD', 'CS_UPC_VRSN_NR_CD', 'SOURCE_CD', 'CAT_NR_CD', 'FMLY_NR_CD', 'ACCT_PLAN_ID', 'DEL_IND', 'NSV_AMT', 'Unnamed: 16'], inplace=True)

In [12]:
bl_full=bl_full_o.copy()

In [13]:
bl_full['CS_UPC_NR_CD']=bl_full['CS_UPC_NR_CD'].astype(str).str.zfill(12)

## Sales

In [14]:
df_sales['WEEK1']=df_sales['WEEK'].str[1:]

In [15]:
df_sales['WEEK1'].unique()

array(['202147', '201930', '202016', '202019', '201906', '202049',
       '201939', '201942', '202051', '201907', '201926', '201949',
       '201904', '201905', '201902', '201903', '201908', '201909',
       '201910', '201911', '201912', '201913', '201914', '201915',
       '201916', '201917', '201918', '201919', '201920', '201921',
       '201922', '201923', '201924', '201925', '201927', '201928',
       '201929', '201931', '201932', '201933', '201934', '201935',
       '201936', '201937', '201938', '201940', '201941', '201943',
       '201944', '201945', '201946', '201947', '201948', '201950',
       '201951', '201952', '202001', '202002', '202003', '202004',
       '202005', '202006', '202007', '202008', '202009', '202010',
       '202011', '202012', '202013', '202014', '202015', '202017',
       '202018', '202020', '202021', '202022', '202023', '202024',
       '202025', '202026', '202027', '202028', '202029', '202030',
       '202031', '202032', '202033', '202034', '202035', '2020

In [16]:
df_sales['CUST_ID_NR_CD'] = df_sales['PA_DIRECT_CD'].astype(str).str.zfill(6)
df_sales['CS_UPC_NR_CD'] = df_sales['CASE_UPC'].astype(str).str.zfill(12)

In [17]:
df_sales=df_sales[['CUST_ID_NR_CD', 'CS_UPC_NR_CD', 'WEEK', 'PHY_CS']]

In [18]:
df_sales=df_sales[df_sales['WEEK'].notna()]

In [19]:
df_sales=df_sales.replace({'CUST_ID_NR_CD': {'.0': ''}})

In [20]:
df_sales['CUST_ID_NR_CD']=df_sales['CUST_ID_NR_CD'].str.zfill(6)

In [21]:
df_sales.head()

,CUST_ID_NR_CD,CS_UPC_NR_CD,WEEK,PHY_CS
0,000668,007940008619,W202147,-0.042
1,000174,003750085930,W201930,-12.000
2,000253,003750000032,W202016,12.000
3,000253,003750000032,W202019,6.000
4,002701,003750000032,W201906,2.000


In [22]:
df_sales['PROMO_PLN_YR_CD']=df_sales['WEEK'].str[1:5].astype(int)
df_sales['WEEK_NR']=df_sales['WEEK'].str[5:].astype(int)

In [23]:
df_sales['WEEK_NR']=df_sales['WEEK'].str[5:].astype(int)

In [24]:
df_sales.columns

Index(['CUST_ID_NR_CD', 'CS_UPC_NR_CD', 'WEEK', 'PHY_CS', 'PROMO_PLN_YR_CD',
       'WEEK_NR'],
      dtype='object')

In [25]:
bl_full['WEEK_NR']=bl_full['WEEK_NR'].astype(int)

In [26]:
bl_full['PROMO_PLN_YR_CD']=bl_full['PROMO_PLN_YR_CD'].astype(int)

In [27]:
df_temp=pd.merge(bl_full, df_sales, how='outer', on=['CUST_ID_NR_CD', 'CS_UPC_NR_CD', 'PROMO_PLN_YR_CD', 'WEEK_NR'])
print(df_sales['PHY_CS'].sum())
print(df_temp['PHY_CS'].sum())
print(df_temp['SUPPLEMENTAL_VOLUME_NR_old'].sum())
print(bl_full['SUPPLEMENTAL_VOLUME_NR_old'].sum())
print(df_temp['SUPPLEMENTAL_VOLUME_NR'].sum())
print(bl_full['SUPPLEMENTAL_VOLUME_NR'].sum())

1255340937.5447512
1255340937.544748
835559845.6102388
835559845.6102388
764207643.1957967
764207643.1957967


In [28]:
df_temp['PROMO_PLN_YR_CD'].unique()

array([2022, 2023, 2021, 2019, 2020], dtype=int64)

In [29]:
df_sales['PROMO_PLN_YR_CD'].unique()

array([2021, 2019, 2020, 2022])

In [30]:
df_temp['IND_ACCT_NR_CD']=df_temp['IND_ACCT_NR_CD'].fillna('000000')

In [31]:
df_temp=df_temp[df_temp['IND_ACCT_NR_CD'].notna()]

In [32]:
df_temp['PROMO_PLN_YR_CD'].unique()

array([2022, 2023, 2021, 2019, 2020], dtype=int64)

## Plans

In [33]:
df_plans.tail()

,PAD,PAI,CASE_UPC,WEEK,PHY_CS,PLAN_UNSV,NSV_PRICE
11264569,003475,000000,085055100596,W202248,0.0,0.00,25.25
11264570,003475,000000,085055100596,W202249,0.0,0.00,25.25
11264571,003475,000000,085055100596,W202250,0.0,0.00,25.25
11264572,003475,000000,085055100596,W202251,0.0,0.00,25.25
11264573,003475,000000,085055100596,W202252,1.0,25.25,25.25


In [34]:
df_plans['CUST_ID_NR_CD'] = df_plans['PAD'].astype(str).str.zfill(6)
df_plans['CS_UPC_NR_CD'] = df_plans['CASE_UPC'].astype(str).str.zfill(12)
df_plans['IND_ACCT_NR_CD']=df_plans['PAI'].astype(str).str.zfill(6)
df_plans['PROMO_PLN_YR_CD']=df_plans['WEEK'].str[1:5]
df_plans['WEEK_NR']=df_plans['WEEK'].str[5:]
df_plans['WEEK_NR']=df_plans['WEEK_NR'].astype(int)
l1=['2021', '2022']
df_plans=df_plans[['CUST_ID_NR_CD', 'IND_ACCT_NR_CD', 'CS_UPC_NR_CD', 'PROMO_PLN_YR_CD', 'WEEK', 'WEEK_NR', 'PHY_CS']]
df_plans.rename(columns={'PHY_CS':'PLANS'}, inplace=True)
df_plans=df_plans[df_plans['PROMO_PLN_YR_CD'].isin(l1)]
df_plans['PROMO_PLN_YR_CD']=df_plans['PROMO_PLN_YR_CD'].astype(int)
# df_temp1=pd.merge(df_temp, df_plans, how='outer', on=['CUST_ID_NR_CD', 'IND_ACCT_NR_CD', 'CS_UPC_NR_CD', 'PROMO_PLN_YR_CD', 'WEEK_NR', 'PHY_CS'])

In [35]:
df_plans['WEEK_NR']=df_plans['WEEK_NR'].astype(int)

In [36]:
k1=df_plans[(df_plans['WEEK']>'W202145')&(df_plans['WEEK']<'W202148')]

In [37]:
k1['PLANS'].sum()

0.0

In [38]:
# df_temp1=pd.merge(df_temp, df_plans, how='outer', on=['CUST_ID_NR_CD', 'IND_ACCT_NR_CD', 'CS_UPC_NR_CD', 'PROMO_PLN_YR_CD', 'WEEK_NR'])

In [39]:
df_plans['WEEK_NR'].unique()

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52])

In [40]:
# df_plans=df_plans[df_plans['PROMO_PLN_YR_CD'].isin(l1)]

In [41]:
 l1=['2021', '2022']
# df_plans=df_plans[df_plans['PROMO_PLN_YR_CD'].isin(l1)]
# df_plans['PROMO_PLN_YR_CD']=df_plans['PROMO_PLN_YR_CD'].astype(int)

In [42]:
df_plans['PROMO_PLN_YR_CD']=df_plans['PROMO_PLN_YR_CD'].astype(int)
df_plans['WEEK_NR']=df_plans['WEEK_NR'].astype(int)

In [43]:
df_temp1=pd.merge(df_temp, df_plans, how='outer', on=['CUST_ID_NR_CD', 'IND_ACCT_NR_CD', 'CS_UPC_NR_CD', 'PROMO_PLN_YR_CD', 'WEEK_NR'])

In [44]:
print(df_plans['PLANS'].sum())
print(df_temp1['PLANS'].sum())

379996959.5599994
379996959.5599996


In [45]:
df_temp1['PROMO_PLN_YR_CD'].unique()

array([2022, 2023, 2021, 2019, 2020], dtype=int64)

In [46]:
df_temp1.head()

,CUST_ID_NR_CD,IND_ACCT_NR_CD,PROMO_PLN_YR_CD,CS_UPC_NR_CD,WEEK_NR,SUPPLEMENTAL_VOLUME_NR,SUPPLEMENTAL_VOLUME_NR_old,WEEK_x,PHY_CS,WEEK_y,PLANS
0,000004,000000,2022,001111100120,2,0.0,22.837979,NaN,NaN,W202202,0.0
1,000004,000000,2022,001111100120,3,0.0,13.930047,NaN,NaN,W202203,0.0
2,000004,000000,2022,001111100120,4,0.0,12.818855,NaN,NaN,W202204,0.0
3,000004,000000,2022,001111100120,5,0.0,12.538084,NaN,NaN,W202205,0.0
4,000004,000000,2022,001111100120,6,0.0,12.538084,NaN,NaN,W202206,0.0


In [47]:
df_temp1['G_WEEK']=df_temp1['PROMO_PLN_YR_CD'].astype(str)+"-"+df_temp1['WEEK_NR'].astype(str)

In [48]:
# import datetime
df_temp1['Date'] = pd.to_datetime(df_temp1['G_WEEK'].add('-1'), format='%Y-%W-%w')


In [49]:
df_temp1['Quarter'] = df_temp1['Date'].dt.quarter

In [50]:
df_temp1.head()

,CUST_ID_NR_CD,IND_ACCT_NR_CD,PROMO_PLN_YR_CD,CS_UPC_NR_CD,WEEK_NR,SUPPLEMENTAL_VOLUME_NR,SUPPLEMENTAL_VOLUME_NR_old,WEEK_x,PHY_CS,WEEK_y,PLANS,G_WEEK,Date,Quarter
0,000004,000000,2022,001111100120,2,0.0,22.837979,NaN,NaN,W202202,0.0,2022-2,2022-01-10,1
1,000004,000000,2022,001111100120,3,0.0,13.930047,NaN,NaN,W202203,0.0,2022-3,2022-01-17,1
2,000004,000000,2022,001111100120,4,0.0,12.818855,NaN,NaN,W202204,0.0,2022-4,2022-01-24,1
3,000004,000000,2022,001111100120,5,0.0,12.538084,NaN,NaN,W202205,0.0,2022-5,2022-01-31,1
4,000004,000000,2022,001111100120,6,0.0,12.538084,NaN,NaN,W202206,0.0,2022-6,2022-02-07,1


## Product hierarchy

In [51]:
df_m=df_m[['Find CaseUPC', 'Category', 'Forecast Group', 'Find Forecast Group']]
# df_m=df_m[['CS_UPC_NR_CD', 'Category', 'Forecast Group', 'Find Forecast Group']]

In [52]:
df_m.rename(columns={'Find CaseUPC':'CS_UPC_NR_CD'}, inplace=True)

In [53]:
df_temp1['IND_ACCT_NR_CD']=df_temp1['IND_ACCT_NR_CD'].astype(str).str.zfill(6)
df_temp1['CS_UPC_NR_CD']=df_temp1['CS_UPC_NR_CD'].astype(str).str.zfill(12)

In [54]:
df_temp2=pd.merge(df_temp1, df_m, how='left', on=['CS_UPC_NR_CD'])

In [55]:
print(df_temp2['PLANS'].sum())
print(df_temp1['PLANS'].sum())

379996959.5599996
379996959.5599996


In [56]:
del df_temp2['WEEK_y']
del df_temp2['WEEK_x']

In [65]:
df_temp2['SUPPLEMENTAL_VOLUME_NR']=df_temp2['SUPPLEMENTAL_VOLUME_NR'].fillna(df_temp2['PHY_CS'])
df_temp2['SUPPLEMENTAL_VOLUME_NR_old']=df_temp2['SUPPLEMENTAL_VOLUME_NR_old'].fillna(df_temp2['PHY_CS'])
df_temp2['PLANS']=df_temp2['PLANS'].fillna(df_temp2['PHY_CS'])

In [ ]:
df_temp2.to_csv('Report PBI.csv', index=False)

## Client hierarchy

In [3]:
df_cust=df_cust[['2.7 Planned Account with Indirect', 'CUST_ID_NR_CD', 'IND_ACCT_NR_CD', 'Customer Group', 'APG']]

In [4]:
df_temp3=pd.merge(df_temp2, df_cust, how='left', on=['CUST_ID_NR_CD', 'IND_ACCT_NR_CD'])

In [5]:
df_temp2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19661599 entries, 0 to 19661598
Data columns (total 15 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   CUST_ID_NR_CD               object 
 1   IND_ACCT_NR_CD              object 
 2   PROMO_PLN_YR_CD             int64  
 3   CS_UPC_NR_CD                object 
 4   WEEK_NR                     int64  
 5   SUPPLEMENTAL_VOLUME_NR      float64
 6   SUPPLEMENTAL_VOLUME_NR_old  float64
 7   PHY_CS                      float64
 8   PLANS                       float64
 9   G_WEEK                      object 
 10  Date                        object 
 11  Quarter                     int64  
 12  Category                    object 
 13  Forecast Group              object 
 14  Find Forecast Group         object 
dtypes: float64(4), int64(3), object(8)
memory usage: 2.2+ GB


In [6]:
df_temp3=df_temp3.fillna(0)

In [7]:
df_temp3=df_temp3.drop_duplicates()

In [8]:
df_temp3['PROMO_PLN_YR_CD'].unique()

array([2022, 2023, 2021, 2019, 2020], dtype=int64)

In [9]:
df_temp3.columns

Index(['CUST_ID_NR_CD', 'IND_ACCT_NR_CD', 'PROMO_PLN_YR_CD', 'CS_UPC_NR_CD',
       'WEEK_NR', 'SUPPLEMENTAL_VOLUME_NR', 'SUPPLEMENTAL_VOLUME_NR_old',
       'PHY_CS', 'PLANS', 'G_WEEK', 'Date', 'Quarter', 'Category',
       'Forecast Group', 'Find Forecast Group',
       '2.7 Planned Account with Indirect', 'Customer Group', 'APG'],
      dtype='object')

In [10]:
df_temp3['Year-week']=df_temp3['PROMO_PLN_YR_CD'].astype(str)+"-"+df_temp3['WEEK_NR'].astype(str).str.zfill(2)

In [14]:
df_temp3.to_csv('Report PBI.csv', index=False)

In [2]:
invest=pd.read_csv('Input-promo-20220111.csv') 
df_temp3 = pd.read_csv('Report PBI.csv', dtype={'CUST_ID_NR_CD':str, 'IND_ACCT_NR_CD':str})

C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Promotions + dispatch rate

In [3]:
del invest['Unnamed: 0']
invest['UPC']=invest['UPC'].astype(str).str.zfill(12)
invest=invest.fillna(0)
invest['CUSTOMER']=invest['cpf'].str[:6]
invest['PROMO_PLN_YR_CD']=invest['G_WEEK'].str[1:5].astype(int)
invest['WEEK_NR']=invest['G_WEEK'].str[5:].astype(int)
invest.rename(columns={'CUSTOMER':'CUST_ID_NR_CD', 'UPC': 'CS_UPC_NR_CD'}, inplace=True)
del invest['cpf']
df1=pd.merge(df_temp3, invest, how='left', on=['CUST_ID_NR_CD', 'CS_UPC_NR_CD', 'PROMO_PLN_YR_CD', 'WEEK_NR'])




In [4]:
df1.to_csv('Report PBI.csv', index=False)

In [2]:
dr=pd.read_csv('dispatch-rate-20220111.csv')
df1 = pd.read_csv('Report PBI.csv', dtype={'CUST_ID_NR_CD':str, 'IND_ACCT_NR_CD':str})

C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,12,13,14,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
dr['CUST_ID_NR_CD']=dr['CUSTOMER'].astype(str).str.zfill(6)
dr['CS_UPC_NR_CD']=dr['UPC']
dr['PROMO_PLN_YR_CD']=dr['G_WEEK'].str[1:5].astype(int)
dr['WEEK_NR']=dr['G_WEEK'].str[5:].astype(int)
dr=dr[['CS_UPC_NR_CD', 'CUST_ID_NR_CD', 'PROMO_PLN_YR_CD', 'WEEK_NR', 'Orders', 'Losses']]
df3=pd.merge(df1, dr, how='left')


MemoryError: 

In [ ]:
df3.to_csv('Report PBI.csv', index=False)

In [ ]:
df3 = pd.read_csv('Report PBI.csv', dtype={'CUST_ID_NR_CD':str, 'IND_ACCT_NR_CD':str})

In [ ]:
df3=df3.fillna(0)
df3['dr']=1+df3['Losses']/df3['Orders']
del df3['G_WEEK_y']
df3.rename(columns={'G_WEEK_x': 'G_WEEK'}, inplace=True)

In [ ]:
df3.to_csv('Report-compare-drp.csv', index=False)